# 1 customized model

In [1]:
import torch
import torch.nn as nn
from peft import LoraConfig, PeftModel, get_peft_model

In [2]:
net = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 2)
)
for name, params in net.named_parameters():
    print(name)


0.weight
0.bias
2.weight
2.bias


In [3]:
config = LoraConfig(target_modules=['0'])
peft_model = get_peft_model(net, config)

In [4]:
peft_model

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=20, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=20, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=20, out_features=2, bias=True)
    )
  )
)

In [11]:
from diffusers import UNet2DConditionModel
net = UNet2DConditionModel(sample_size=64)
for name, params in net.named_parameters():
    if 'to' in name:
        print(name)

down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_out.0.bias
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_q.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_k.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_v.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.weight
down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_out.0.bias
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_q.weight
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_k.weight
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_v.weight
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_out.0.weight
down_blocks.0.attentions.1.transformer_blocks.0.attn1.to_out

In [12]:
unet_lora_config = LoraConfig(r=8, init_lora_weights="gaussian", target_modules=["to_k", "to_q", "to_v", "to_out.0"])
peft_model = get_peft_model(net, unet_lora_config)

In [13]:
peft_model.print_trainable_parameters()

trainable params: 1,725,440 || all params: 874,025,924 || trainable%: 0.19741290877317272


# 2 multi-lora switch

## create 2 lora modules

In [33]:
net = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 2)
)

In [34]:
config0 = LoraConfig(target_modules=['0'])
model0 = get_peft_model(net, config0)
model0.save_pretrained("./lora0")


In [35]:

config2 = LoraConfig(target_modules=['2'])
model2 = get_peft_model(net, config2)
model2.save_pretrained("./lora2")


In [36]:
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=20, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=20, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=20, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=20, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=2, bias=False)
        )
        (lora_embedding_A):

## load

In [40]:
net = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 2)
)
lora_model = PeftModel.from_pretrained(net, "./lora0", adapter_name="lora0")
lora_model

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=20, bias=True)
        (lora_dropout): ModuleDict(
          (lora0): Identity()
        )
        (lora_A): ModuleDict(
          (lora0): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (lora0): Linear(in_features=8, out_features=20, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=20, out_features=2, bias=True)
    )
  )
)

In [41]:
lora_model.load_adapter("./lora2", adapter_name="lora2")

_IncompatibleKeys(missing_keys=['base_model.model.0.base_layer.weight', 'base_model.model.0.base_layer.bias', 'base_model.model.0.lora_A.lora0.weight', 'base_model.model.0.lora_B.lora0.weight', 'base_model.model.2.base_layer.weight', 'base_model.model.2.base_layer.bias'], unexpected_keys=['base_model.model.0.lora_A.lora2.weight', 'base_model.model.0.lora_B.lora2.weight'])

In [44]:
for name, params in lora_model.named_parameters():
    print(name)

base_model.model.0.base_layer.weight
base_model.model.0.base_layer.bias
base_model.model.0.lora_A.lora0.weight
base_model.model.0.lora_B.lora0.weight
base_model.model.2.base_layer.weight
base_model.model.2.base_layer.bias
base_model.model.2.lora_A.lora2.weight
base_model.model.2.lora_B.lora2.weight


In [45]:
lora_model.active_adapter

'lora0'

In [46]:
lora_model.set_adapter("lora2")
lora_model.active_adapter

'lora2'

In [51]:
for name, params in lora_model.named_parameters():
    print(name, params)

base_model.model.0.base_layer.weight Parameter containing:
tensor([[ 0.0604,  0.0137, -0.2185,  0.1296, -0.0280,  0.2921,  0.1835, -0.0641,
          0.1377,  0.2154],
        [ 0.0983,  0.0188, -0.2438, -0.2738,  0.1732,  0.2957,  0.2143,  0.1302,
          0.0398,  0.2023],
        [-0.0704,  0.1617,  0.0558,  0.2920,  0.1776, -0.1266, -0.2570,  0.1677,
         -0.2258, -0.1908],
        [ 0.0992,  0.2700, -0.2348, -0.1692,  0.2895, -0.2877,  0.0026, -0.1235,
         -0.1730, -0.2905],
        [-0.0129, -0.0434,  0.0714, -0.2570,  0.2909,  0.0829,  0.0987, -0.1282,
          0.0068, -0.0956],
        [-0.0539, -0.0901,  0.0095,  0.2184, -0.0552,  0.3044,  0.1577,  0.0090,
         -0.0753,  0.2160],
        [ 0.1120,  0.1791,  0.0638,  0.0354, -0.2394,  0.2979, -0.1689,  0.0792,
          0.0664,  0.1228],
        [ 0.2668,  0.0697, -0.0941, -0.1233,  0.2935, -0.1532,  0.1243,  0.0610,
          0.1191, -0.2520],
        [-0.1284, -0.2475, -0.1974,  0.2049,  0.1029,  0.2042, -0.063

# 3 diable lora

In [54]:
x = torch.randn(1, 10)
net(x)

tensor([[-0.1976,  0.2378]], grad_fn=<AddBackward0>)

In [55]:
with lora_model.disable_adapter():
    print(lora_model(x))

tensor([[-0.1976,  0.2378]])
